<a href="https://colab.research.google.com/github/VivekSaini11/AI-Capstone-Project/blob/master/Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DESCRIPTION**

**Problem Statement**

Finance Industry is the biggest consumer of Data Scientists. It faces constant attack by fraudsters, who try to trick the system. Correctly identifying fraudulent transactions is often compared with finding needle in a haystack because of the low event rate. 
It is important that credit card companies are able to recognize fraudulent credit card transactions so that the customers are not charged for items that they did not purchase.
You are required to try various techniques such as supervised models with oversampling, unsupervised anomaly detection, and heuristics to get good accuracy at fraud detection.
Dataset Snapshot

The datasets contain transactions made by credit cards in September 2013 by European cardholders. This dataset represents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

In [ ]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("train_data.csv")
train

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,38355.0,1.043949,0.318555,1.045810,2.805989,-0.561113,-0.367956,0.032736,-0.042333,-0.322674,0.499167,-0.572665,0.346009,-0.047407,-0.098964,-0.663284,0.181411,-0.124345,-0.790453,-0.720944,-0.084556,-0.240105,-0.680315,0.085328,0.684812,0.318620,-0.204963,0.001662,0.037894,49.67,0.0
1,22555.0,-1.665159,0.808440,1.805627,1.903416,-0.821627,0.934790,-0.824802,0.975890,1.747469,-0.658751,1.281502,-1.430087,0.372028,1.403024,-2.739413,-1.331766,1.964590,-0.205639,1.325588,-0.373759,-0.335332,-0.510994,0.035839,0.147565,-0.529358,-0.566950,-0.595998,-0.220086,16.94,0.0
2,2431.0,-0.324096,0.601836,0.865329,-2.138000,0.294663,-1.251553,1.072114,-0.334896,1.071268,-1.109522,-1.016020,-0.654945,-1.473470,0.317345,1.067491,-0.372642,-0.674725,0.369841,0.095583,-0.039868,0.012220,0.352856,-0.341505,-0.145791,0.094194,-0.804026,0.229428,-0.021623,1.00,0.0
3,86773.0,-0.258270,1.217501,-0.585348,-0.875347,1.222481,-0.311027,1.073860,-0.161408,0.200665,0.154307,0.882673,0.547890,0.269484,-1.253302,-0.883963,0.495221,-0.153212,0.296710,0.136148,0.382305,-0.424626,-0.781158,0.019316,0.178614,-0.315616,0.096665,0.269740,-0.020635,10.78,0.0
4,127202.0,2.142162,-0.494988,-1.936511,-0.818288,-0.025213,-1.027245,-0.151627,-0.305750,-0.869482,0.428729,1.136666,0.273476,0.697123,-1.222134,-0.938820,1.298149,0.912921,-0.793721,1.064984,0.106592,0.010115,0.021722,0.079463,-0.480899,0.023846,-0.279076,-0.030121,-0.043888,39.96,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28740,62600.0,-0.573875,0.698454,1.664781,0.271280,-0.271413,-0.185920,0.600426,0.126789,0.138870,-0.518334,-0.488966,0.251363,-0.396139,-0.219544,-0.409363,-0.875436,0.499299,-0.903744,-0.162195,0.087849,0.046492,0.445377,-0.036869,0.445691,-0.245296,0.371932,0.363133,0.202502,39.99,0.0
28741,49008.0,1.184948,-0.659465,1.555054,0.709977,-1.896539,-0.451101,-1.178725,0.121698,-0.253546,0.824531,-0.636419,-0.663363,-0.727632,-0.055607,1.499873,-0.735345,-0.336759,1.983360,-1.730073,-0.536503,-0.083321,0.158833,0.041396,0.643889,0.156078,-0.256274,0.088038,0.054656,40.00,0.0
28742,53217.0,0.989208,-1.436164,1.296648,-0.169846,-1.746214,0.495077,-1.232567,0.224558,0.387741,0.173004,-0.705684,0.267736,0.641486,-1.278141,-0.917692,0.322377,1.067970,-1.922918,0.552313,0.349327,0.214345,0.636191,-0.165064,0.166475,0.337568,-0.106958,0.066535,0.050619,156.28,0.0
28743,68109.0,-0.920600,-5.154184,-1.063193,-1.836148,-2.509258,0.515280,0.419332,-0.124824,1.431024,-1.315644,-0.241888,0.906537,-0.198501,0.017713,0.079024,-2.089884,0.026858,2.442484,1.050970,1.941292,0.179847,-1.396656,-1.182621,-0.805502,0.155312,-0.907921,-0.102587,0.223225,1214.35,0.0


In [ ]:
test = pd.read_csv('test_data.csv') 
test

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,113050.0,0.114697,0.796303,-0.149553,-0.823011,0.878763,-0.553152,0.939259,-0.108502,0.111137,-0.390521,-1.949546,-0.494436,-0.353696,0.158729,-0.267239,0.234802,-0.754936,-0.343012,0.312175,-0.042711,-0.335776,-0.807853,-0.055940,-1.025281,-0.369557,0.204653,0.242724,0.085713,0.89
1,26667.0,-0.039318,0.495784,-0.810884,0.546693,1.986257,4.386342,-1.344891,-1.743736,-0.563103,-0.616315,-0.587786,0.317419,-0.408521,0.719639,0.226926,-0.296316,-0.040143,0.119177,1.057682,0.926255,-1.377003,-0.072200,-0.197573,1.014807,1.011293,-0.167684,0.113136,0.256836,85.00
2,159519.0,2.275706,-1.531508,-1.021969,-1.602152,-1.220329,-0.462376,-1.196485,-0.147058,-0.950224,1.560463,-1.753256,-1.331010,-0.061941,-0.405532,0.048083,-0.307503,0.289363,0.189739,0.022546,-0.408289,-0.193271,-0.103533,0.150945,-0.811083,-0.197913,-0.128446,0.014197,-0.051289,42.70
3,137545.0,1.940137,-0.357671,-1.210551,0.382523,0.050823,-0.171322,-0.109124,-0.002115,0.869258,-0.001965,0.607629,1.048673,-0.514821,0.329538,-1.041463,-0.498797,-0.276887,0.114245,0.379447,-0.199280,0.157994,0.650355,0.034206,0.739535,0.223605,-0.195509,-0.012791,-0.056841,29.99
4,63369.0,1.081395,-0.502615,1.075887,-0.543359,-1.472946,-1.065484,-0.443231,-0.143374,1.659826,-1.131238,0.173132,1.430172,0.915609,-0.336588,1.140171,-0.653626,-0.016567,0.066287,0.242537,0.059880,0.224157,0.821209,-0.137223,0.986259,0.563228,-0.574206,0.089673,0.052036,68.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28839,147732.0,-9.953085,-8.868956,-3.048033,0.116059,-1.944722,1.698227,2.030621,0.418490,-0.427001,-0.455292,0.757697,0.295987,0.449608,0.093262,0.032205,1.952463,0.594412,-2.434499,-0.067459,-2.011904,-0.927693,-0.985009,-1.748027,-1.332942,0.172809,-0.538229,-0.547308,0.681201,774.82
28840,121072.0,-0.635801,1.440920,-0.716306,-0.772760,0.595685,-0.530460,1.035374,0.018418,0.242614,0.230560,1.208943,0.651500,-0.046371,-1.041335,-0.896959,0.449309,0.123550,0.183364,-0.077676,0.365754,-0.392902,-0.755853,0.151727,0.654259,-0.303251,0.103183,0.567191,0.257921,47.68
28841,45542.0,-2.739001,2.245648,1.170273,-2.182795,-0.000674,-0.345733,1.080504,-0.458405,2.553908,3.655100,1.861208,-0.094565,-1.275883,-1.221912,0.453803,-0.014116,-1.097329,-0.455590,-1.067691,1.660038,-0.543367,-0.181322,-0.062157,-0.022646,-0.020118,0.670730,0.760675,-0.063545,4.53
28842,67408.0,-1.071097,-0.190828,2.922398,-1.561072,-1.399093,0.908170,-1.039460,0.783481,0.408617,0.169271,0.199793,-0.811578,-2.436921,-0.985158,-2.139829,0.685896,0.818486,-0.979143,0.381122,0.139891,0.162669,0.809700,-0.358580,0.023290,0.424569,-0.088501,0.451128,0.171191,1.50


In [ ]:
test_hidden = pd.read_csv('test_data_hidden.csv') 
test_hidden

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,113050.0,0.114697,0.796303,-0.149553,-0.823011,0.878763,-0.553152,0.939259,-0.108502,0.111137,-0.390521,-1.949546,-0.494436,-0.353696,0.158729,-0.267239,0.234802,-0.754936,-0.343012,0.312175,-0.042711,-0.335776,-0.807853,-0.055940,-1.025281,-0.369557,0.204653,0.242724,0.085713,0.89,0.0
1,26667.0,-0.039318,0.495784,-0.810884,0.546693,1.986257,4.386342,-1.344891,-1.743736,-0.563103,-0.616315,-0.587786,0.317419,-0.408521,0.719639,0.226926,-0.296316,-0.040143,0.119177,1.057682,0.926255,-1.377003,-0.072200,-0.197573,1.014807,1.011293,-0.167684,0.113136,0.256836,85.00,0.0
2,159519.0,2.275706,-1.531508,-1.021969,-1.602152,-1.220329,-0.462376,-1.196485,-0.147058,-0.950224,1.560463,-1.753256,-1.331010,-0.061941,-0.405532,0.048083,-0.307503,0.289363,0.189739,0.022546,-0.408289,-0.193271,-0.103533,0.150945,-0.811083,-0.197913,-0.128446,0.014197,-0.051289,42.70,0.0
3,137545.0,1.940137,-0.357671,-1.210551,0.382523,0.050823,-0.171322,-0.109124,-0.002115,0.869258,-0.001965,0.607629,1.048673,-0.514821,0.329538,-1.041463,-0.498797,-0.276887,0.114245,0.379447,-0.199280,0.157994,0.650355,0.034206,0.739535,0.223605,-0.195509,-0.012791,-0.056841,29.99,0.0
4,63369.0,1.081395,-0.502615,1.075887,-0.543359,-1.472946,-1.065484,-0.443231,-0.143374,1.659826,-1.131238,0.173132,1.430172,0.915609,-0.336588,1.140171,-0.653626,-0.016567,0.066287,0.242537,0.059880,0.224157,0.821209,-0.137223,0.986259,0.563228,-0.574206,0.089673,0.052036,68.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30650,116581.0,2.067965,0.221921,-1.642460,0.433789,0.440495,-0.919987,0.245535,-0.291363,0.358657,-0.403301,-0.331212,0.631121,0.893351,-1.006748,0.104773,0.246664,0.462542,-0.445184,0.008261,-0.104583,-0.347978,-0.856428,0.342183,0.661849,-0.243279,0.162404,-0.057066,-0.027550,1.98,0.0
30651,127147.0,1.958020,0.084885,-1.809655,0.539936,0.239498,-1.402936,0.399516,-0.445609,0.477641,-0.520829,-0.348767,0.428035,0.804493,-0.878217,0.831005,0.216808,0.300546,0.464709,-0.400390,-0.036076,0.229328,0.747422,-0.098217,-0.150026,0.278139,-0.102982,-0.008284,-0.021019,60.51,0.0
30652,140658.0,1.034519,-1.657062,-2.523365,0.511050,0.457808,0.892110,0.355369,0.200559,1.163889,-1.154819,-0.400307,-0.245489,-1.269597,-1.047925,0.897011,-0.642808,1.836402,-0.932019,-0.934731,0.623857,0.070859,-0.572146,-0.131130,-0.689302,-0.423416,-0.055265,-0.057818,0.049330,472.42,0.0
30653,133603.0,1.176521,-1.500626,-0.766906,1.473935,-0.854656,0.064217,-0.114347,0.045600,0.748702,0.235806,0.347142,0.402140,-0.676666,0.435125,0.239739,0.840228,-1.096658,1.127104,-0.472692,0.519312,0.484340,0.584323,-0.250906,-0.354208,-0.249364,-0.600547,-0.027085,0.023616,400.00,0.0


In [ ]:
# Check all the latent features and parameters with their mean and standard deviation. Value are close to 0 centered (mean)
 # with unit standard deviation
train.std(axis = 0, skipna = True) 

Time      47622.292902
V1            2.055007
V2            1.752448
V3            1.569294
V4            1.446612
V5            1.404393
V6            1.333488
V7            1.289288
V8            1.253515
V9            1.123971
V10           1.141996
V11           1.030977
V12           1.029904
V13           1.000798
V14           0.992349
V15           0.916225
V16           0.901976
V17           0.903369
V18           0.845332
V19           0.817528
V20           0.829378
V21           0.784600
V22           0.732039
V23           0.688742
V24           0.601607
V25           0.524333
V26           0.484919
V27           0.408254
V28           0.345624
Amount      271.157774
Class         0.043702
dtype: float64

In [ ]:
# Check the class count for each class. It’s a class Imbalance problem.
train['Class'].value_counts().plot(kind="bar")

In [ ]:
# Check the class count for each class. It’s a class imbalance problem.
train['Class'].value_counts()

0.0    76513
1.0      130
Name: Class, dtype: int64

In [ ]:
train.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [ ]:
train = train.fillna(0)
train

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,38355.0,1.043949,0.318555,1.045810,2.805989,-0.561113,-0.367956,0.032736,-0.042333,-0.322674,0.499167,-0.572665,0.346009,-0.047407,-0.098964,-0.663284,0.181411,-0.124345,-0.790453,-0.720944,-0.084556,-0.240105,-0.680315,0.085328,0.684812,0.318620,-0.204963,0.001662,0.037894,49.67,0.0
1,22555.0,-1.665159,0.808440,1.805627,1.903416,-0.821627,0.934790,-0.824802,0.975890,1.747469,-0.658751,1.281502,-1.430087,0.372028,1.403024,-2.739413,-1.331766,1.964590,-0.205639,1.325588,-0.373759,-0.335332,-0.510994,0.035839,0.147565,-0.529358,-0.566950,-0.595998,-0.220086,16.94,0.0
2,2431.0,-0.324096,0.601836,0.865329,-2.138000,0.294663,-1.251553,1.072114,-0.334896,1.071268,-1.109522,-1.016020,-0.654945,-1.473470,0.317345,1.067491,-0.372642,-0.674725,0.369841,0.095583,-0.039868,0.012220,0.352856,-0.341505,-0.145791,0.094194,-0.804026,0.229428,-0.021623,1.00,0.0
3,86773.0,-0.258270,1.217501,-0.585348,-0.875347,1.222481,-0.311027,1.073860,-0.161408,0.200665,0.154307,0.882673,0.547890,0.269484,-1.253302,-0.883963,0.495221,-0.153212,0.296710,0.136148,0.382305,-0.424626,-0.781158,0.019316,0.178614,-0.315616,0.096665,0.269740,-0.020635,10.78,0.0
4,127202.0,2.142162,-0.494988,-1.936511,-0.818288,-0.025213,-1.027245,-0.151627,-0.305750,-0.869482,0.428729,1.136666,0.273476,0.697123,-1.222134,-0.938820,1.298149,0.912921,-0.793721,1.064984,0.106592,0.010115,0.021722,0.079463,-0.480899,0.023846,-0.279076,-0.030121,-0.043888,39.96,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76639,155718.0,-5.043373,4.876364,-2.702027,-1.446244,-0.903016,-1.632328,0.481223,0.524592,3.792566,5.843843,-0.683398,0.000348,-0.392878,-0.714283,0.804899,-0.830393,-0.516421,-0.084422,-0.505206,2.287963,-0.365556,1.078138,0.082662,-0.007839,0.025701,-0.381600,0.163275,-0.974517,0.99,0.0
76640,46255.0,-4.962864,-5.266797,1.091004,-1.317072,1.200263,-1.455706,-2.095754,-0.072299,-1.564920,1.659980,0.957131,-0.689424,-0.052090,-0.406384,0.427602,0.333786,0.027511,0.433809,-0.722314,-1.536044,-0.409620,-0.022736,-2.618592,-0.081345,-0.269031,-0.251214,0.286546,-1.448318,138.00,0.0
76641,1285.0,-3.835659,-1.955510,-0.404067,0.438873,1.787822,-1.066112,-0.275514,0.221039,0.083869,0.693806,0.257107,0.911536,1.116799,-0.058543,-0.113617,0.166827,-0.436120,0.201192,2.078902,-1.180631,-0.661170,0.285619,1.336996,-0.688062,0.784588,1.281218,0.224189,-0.883502,14.50,0.0
76642,45263.0,-1.030194,1.115501,0.261893,0.205956,1.058084,0.500253,1.061548,0.164613,-1.064402,-0.070975,0.505182,0.922937,1.025008,0.444673,-0.022597,0.103646,-0.770751,-0.182560,0.401193,-0.175221,-0.226457,-0.562242,0.167762,-1.188438,-0.309820,-0.749035,-0.354559,-0.135593,59.30,0.0


In [ ]:
train.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [ ]:
#  Oversampling or undersampling can be used to tackle the class imbalance problem
print(Counter(train['Class']))
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(train.drop(['Class'], axis=1), train['Class'])
X_test_over, y_test_over = oversample.fit_resample(test_hidden.drop(['Class'], axis=1), test_hidden['Class'])
print(Counter(y_over))

Counter({0.0: 76514, 1.0: 130})
Counter({0.0: 76514, 1.0: 76514})


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_over = pca.fit_transform(X_over)
X_over

NameError: ignored

In [ ]:
# Use Tree-based classifiers like Random Forest and XGBoost.
regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_over, y_over)
y_pred_regressor = regressor.predict(test_tfidf)
y_pred_regressor[:10]

In [ ]:
# precision, recall, F1-score, AUC-ROC curve. Use F1-Score for SVM
accuracy = accuracy_score(y_test_over, y_pred_svm)
print('Accuracy: %f' % accuracy)
precision = precision_score(y_test_over,  y_pred_svm, average='micro')
print('Precision: %f' % precision)
recall = recall_score(y_test_over,  y_pred_svm, average='micro')
print('Recall: %f' % recall)
f1 = f1_score(y_test_over,  y_pred_svm, average='micro')
print('F1 score: %f' % f1)